# The Fruithunter pipeline

## Overview

The fruithunter pipeline was developed to segment and count apples from point clouds resulting of scans of apples orchards. Different measurement protocols were considered for orchard scans. As illustrated in figure 1, two main protocol were considered. In the first one, scans are positionned every five trees in the middle of the surounding rows. In the second protocol, scans are performed from close positions in the left and right rows of each tree. The protocols are named low and high resolution protocols respectivelly.

<figure>
    <img src="imgs/protocols.png"
         alt="Protocols" 
         width="400" 
         height="500">
    <figcaption>Figure 1. Field measurement protocols schematic. Green circles labeled with a "T" represent the apple trees. Gray circles labelled with an "S" represent the sensor position at the moment of the measurement. The low and hight resolution protocol are presented in the left and right part of the figure respectivelly. </figcaption>
</figure>

Each protocol set different levels of precision in the description of the apples trees. In the low resolution protocol, a gradient of densities can be observed that show decrease of density from the scan positions to trees at longest distances from the scan positions as shown in Figure 2. In this figure, density gradient is represented by colors with blue representing lowest density and red highest one. In the high resolution protocol, all trees have similar point densities. The scans contains different levels of noise due to different phenomena. First, occlusion between elements create missing data. Second, outliers are created from multiple hits from the same lidar beam creating intermediate points. Environmental conditions (wind, lighting) can also create perturbation in the measurement.

By default, X,Y,Z coordinates are infered from Lidar scans but other radiometric features can be considered. In our case, from real scans, reflectance, deviation and amplitude of the signal can be considered for each points. 

<figure>
    <img src="imgs/density.png"
         alt="Protocols" 
         width="400" 
         height="500">
    <figcaption>Figure 2. Point cloud density for the low resolution protocol</figcaption>
</figure>

From this noisy data set, it is not possible to define a simple set of rules to segment apples. To handle this complex signal, different approaches from machine and deep learning are going to be used. The first one will use random forest as a segmentation module and will use the FPFH as a feature descriptor. The second approach will use RandLA-NET, a deep learning model specially designed to consume point clouds with millions of points  and make segmentation of multiple elements in a efficient way. The global workflow of the pipeline could bee see it on figure 3. The Random Forest and RandLA-NET models will be detailled in the following sections.

<figure>
    <img src="imgs/fruithunter.png"
         alt="Fruithunter Pipeline" 
         width="600" 
         height="700">
    <figcaption>Figure 3. The fruithunter pipeline</figcaption>
</figure>

### Random Forest 

Random Forest is an algorithm composed by <b>N</b> blocks of decision trees. A decision tree will use a feature map and will generate a series of questions to infer the class labels of the sampled points. The idea behind this procedure is to split the data on the features that returns the biggest values for the information gain (IG). The feature splitting will continue until each final node belongs to the same class. To avoid over hitting of the model, decision trees are pruned according to a predefined maximal depth.  The idea of using several decision trees is to minimise the variance of the model without increasing the bias. In general, random forest are considered to have good classification perfomance and scalability. 

In this project random forests are fitted with 33 features (FPFH) or 36 if the radiometric features are added, the model is composed by 300 decision trees and the depth of each decision tree is 15 nodes. 

### RandLA-NET

RandLA-NET is a state-of-the-art deep learning model developed to consume raw point clouds of large size (>1M) and perform segmentation with good accuracy. In order to achieve this objective, 3 new layers were developed as illustrated in figure 4.   

<figure>
    <img src="imgs/rdnet_layers.png"
         alt="rdnet layers" 
         width="600" 
         height="700">
    <figcaption>Figure 4. RandLA-NET main Layers</figcaption>
</figure>

The <i>local spatial encoding</i> is applied on each point and determine and encode the relative XYZ coordinates of all the neighbouring points with additionnal point features. Particularly this layer allows the network to abstract complex geometries. The <i>Attentive Pooling</i> layer is used to aggregate neighbouring point features using attention score for each feature. At the end these two layers will generate a informative feature vector with the most representative features and points. Finally the <i>Dilated residual block</i> will be in charge of stacking and propagating the features vector to neighboring points to increasing the receptive field that represent each point. A graphical representation of this process could be see it in the figure 5. 

<figure>
    <img src="imgs/receptiveFields.png"
         alt="rdnet layers" 
         width="400" 
         height="500">
    <figcaption>Figure 5. Dilated residual block, Global feature extraction</figcaption>
</figure>


### Clustering 

To count the number of fruits, a clustering algorithm is used. The <i>DBSCAN</i> algorithm was chosen as it groups the segmented points in different clusters based on spatial density. 

DBSCAN is parameterized with two main elements: a distance threshold (<i>eps</i>) that specifies how close should be a group of points to be in the same cluster, and a minimum number of points to consider a region as dense.  



## Installation 

To install the fruithunter pipeline and its dependencies must be followed the foollowing steps:

<ol>
    <li> Create a virtual environment with <a href="https://docs.conda.io/">conda</a> :<br>

```bash
conda create -n fruithunter  -c conda-forge -c anaconda python=3.6 scikit-learn scipy numpy pytest jupyter pandas matplotlib pcl cmake eigen boost tensorflow-gpu=1.13.1 PyYAML=5.1.2 cython=0.29.15 h5py=2.10.0
```
      
  <li> Activate the conda environment <br>

```bash
conda activate fruithunter
```
      
  <li> Install final dependency of Randla-NET <br>

```bash
pip install open3d-python==0.3.0
```
      
  <li> Retrieve the source code of fruithunter:

```bash
git clone https://forgemia.inra.fr/openalea_phenotyping/fruithunter.git
cd fruithunter/
```

  <li> Compile the package for the feature extraction (FPFH):
      

   - Create the build folder: <br>

```bash
mkdir pcl/build
```
   - Generate the cmake file: <br>

```bash
cd pcl/build; cmake .. ; make ; cd -
```
  <li> Get the Randla-NET repository: <br>

```bash
    git clone https://github.com/jrojas9206/RandLA-Net.git randlanet
```
     
  <li> Compile the wrappers for Randla-NET:

```bash
cd randlanet/utils/cpp_wrappers ; sh compile_wrappers.sh ; cd -
```
      
  <li> Install the nearest neighbors module:
      
```bash
cd randlanet/utils/nearest_neighbors/ ; python setup.py install ; cd -
```
  <li> Install the pcl and randlanet modules:

```bash
python setup.py develop
```
      
<ol>

## Generic definitions 
In order to follow a simple order in this notebook, a generic python library will be imported and some paths to the data to be processed will be defined. Look the next cells.

In [ ]:
# Import the generic python libraries  with the following purposes 
# -> Handle the files
import os
import glob 
# -> Handle the arrays that represents the point clouds
import numpy as np
# -> To plot few interesting behaviours 
import matplotlib.pyplot as plt

In [ ]:
# Path to datasets 
data2annotatedApples = "data/example2notebook_realdata/"
data_fpfh = os.path.join(data2annotatedApples, "fpfh")
dataRDnet = os.path.join(data2annotatedApples, "data2randlanet")
classicSeg= os.path.join(data2annotatedApples, "classicSegmentation_fpfh")

In [ ]:
def split_path_and_fileName(strPathName):
    """
    Split the path and the name of the file of the given string 
    :INPUT:
        strPathName: str with the path and name of a file. ex: /data/pointcloud.txt
    :OUTPUT:
        list, ["/data/", "pointcloud.txt"]
    """
    for idx in range(len(strPathName)-1, 0, -1):
        if(strPathName[idx]=='/'):
            return strPathName[0:idx], strPathName[idx+1:]
    return None

In [ ]:
from sklearn.model_selection import train_test_split
from randlanet.utils.data_prepare_apple_tree import * 

def split_dataSet(path2files, path2output, model, verbose=False, protocol="field"):
    """
    Split and prepare the data on test and train for the implemented algorithms 
    Random Forest and RandLA-NET and 
    :INPUT:
        path2files : str of the path to the folder of input files
        path2output: str of the path to the output folder 
        model      : str, "rdf" or "rdnet"
        verbose    : If true print few message of the code steps 
        protocol   : Type of protocol to handle ; synthetic/field/field_only_xyz
    :OUTPUT:
        Write the splitted dataset  on the folder
    """
    # NOTE: This segment will be only executed from the notebook 
    lstOfFiles = glob.glob(os.path.join(path2files,"*.txt"))
    if(verbose):
        print("Found files: %i " %(len(lstOfFiles)))
    # Split the files
    X_train, X_test, _,_ = train_test_split(lstOfFiles, range(len(lstOfFiles)), test_size=0.20, random_state=42)
    if(verbose):
        print(" -> Train set: %i" %len(X_train))
        print(" -> Test set : %i" %len(X_test))
    # Create the directory to keep the test and train sets 
    path2initialSplit = path2output #os.path.join(data2annotatedApples, "dataToRDF")
    if(not os.path.isdir(path2initialSplit)):
        os.mkdir(path2initialSplit)
    for folderName, fileList in zip( ["train" if model == "rdf" else "training", "test"], [X_train, X_test] ):
        path2saveData = os.path.join(path2initialSplit)
        for file2feature in fileList:
            output2wrt = os.path.join(path2saveData, folderName)
            if(not os.path.isdir(output2wrt)):
                os.mkdir(output2wrt)
                print("Folder was created: %s" %output2wrt)
            print("-> Loading: %s" %split_path_and_fileName(file2feature)[1])
            file2wrt = os.path.join(output2wrt, split_path_and_fileName(file2feature)[1])
            if(model == "rdf"):
                # NOTE: If you change the position or the name of the feature generator change the
                # next string "cmd2feature" [execution command]
                cmd2features = "./pcl/build/my_feature %s %.3f %s %s" %("fpfh",          # Feature extractor 
                                                                        0.025,           # Grid size 
                                                                        file2feature,    # Input File
                                                                        file2wrt)        # Output File
                print(" -> Running feature extractor")
                os.system(cmd2features)
            else: # RandLA-NET
                if(folderName=="test"):
                    convert_for_test(file2feature, path2saveData, grid_size=0.001, protocol=protocol)
                else:
                    convert_for_training(file2feature, None, path2saveData, grid_size=0.001, protocol=protocol)

## Preparing the data 
The algorithms used, Random Forest and RandLA-NET, consume different representations of the point clouds and read different formats. Random Forest will use the features obtained from the FPFH to find the set of rules that gives the best classification of the apple's points. This script will load the point clouds from the "txt" files, and expect two type of order in the data for the training process, the first is XYZ+Annotations+FPFH, the second one is XYZ+Rediometric Features+Annotation+FPFH. And for the class prediction process the algorithm expects just the XYZ+FPFH. To get the desired features, two types of approaches can be followed.

The first way to do this is by running the main script that generate the FPFH features. To run it, you have to locate the path "fruithunter/pcl" and inside of this folder execute the script called "launcher.py". An example of how execute this script is shown below.

In [ ]:
# This command will execute the python script, the first argument is the path where the point clouds are located 
# and the second argument is the folder where you want to put the output files. 
# Note 1: The below command was executed on the field measurements 
# Note 2: The following command will be executed as if it were executed from the terminal 
!python pcl/launcher.py data/example2notebook_realdata/ data/example2notebook_realdata/fpfh/

In [ ]:
# Note: We will execute the same command of above but to prepare the synthetic data 
!python pcl/launcher.py data/example2notebook_synthetic/ data/example2notebook_synthetic/fpfh/

If you want to execute the FPFH feature extraction after do some pre-processing you can do it by importing the respective module. In the following cell is shown the general code flow.

In [ ]:
# Import the module that contain the FPFH feature extractor 
from pcl.launcher import launch_feature 
# Call the final method of the feature generation 
launch_feature(data2annotatedApples, data_fpfh)# This method will look for the txt files in the input path
                                               # 'data2annotatedApples' It will write the featured files on 
                                               # the path 'data_fpfh'

Now if you verify the output path, you will see that new files have been created after execute the previous cells. This new files have the FPFH features. To see the differences between the files, execute the next cell.

In [ ]:
# Lists with the names of the input and output files of the previous cells 
inputFiles  = glob.glob(os.path.join(data2annotatedApples,"*.txt"))
outputFiles = glob.glob(os.path.join(data2annotatedApples, "fpfh","*.txt")) 
lst_idx     = 0
# Load a file and show their shape
shape_inputFile = np.loadtxt(inputFiles[lst_idx]).shape
shape_outputFile= np.loadtxt(outputFiles[lst_idx]).shape
print("Input files: %i | Output files: %s" %(len(inputFiles), len(outputFiles)))
print("Shape input file: %s | Shape output file: %s" %(str(shape_inputFile), str(shape_outputFile)))
print("Size input file: %i[bytes] | Size output file: %s[bytes]" %(os.path.getsize(inputFiles[lst_idx]), os.path.getsize(outputFiles[lst_idx])))

In the case of randlanet the <i>txt</i> files have to be converted to <i>ply</i> format and their <i>kdtree</i> must be defined. On the other hand the model will consume the raw points (XYZ) or (XYZ+radiometric features). To clarify, in this case there is no need to estimate an internal vector of characteristics because the model finds the best characteristics that define the geometries by itself.<br>

To do the file conversion could be followed two different ways. You can run the script "utils/data_prepare_apple_tree.py" or call the method on your own code. To see the process look the next cells.

In [ ]:
# Executing the main script from terminal -- Training data 
!python utils/data_prepare_apple_tree.py data/example2notebook_realdata/toRandlanet/training/ --outputDir=data/example2notebook_realdata/toTrainRandLA_NET/training/ --datasetType=train

In [ ]:
# Executing the main script from terminal -- Test data
# Note that when you set the argument datasetType to test the annotations in the outputfile will not appear 
!python utils/data_prepare_apple_tree.py data/example2notebook_realdata/toRandlanet/test/ --outputDir=data/example2notebook_realdata/toTrainRandLA_NET/ --datasetType=test

In [ ]:
# Executing the above task in your own code 

# Import the methods
from randlanet.utils.data_prepare_apple_tree import * 
# We will execute the same behaviour of above but in the synthetic data --
# for this we will create an extra variable
path2Sdata         = "data/example2notebook_synthetic/"
path2inputTraining = os.path.join(path2Sdata, "toRandlanet/training/")
path2inputTest     = os.path.join(path2Sdata, "toRandlanet/test/")
path2outpuT        = os.path.join(path2Sdata, "toTrainRandLA_NET/")
# Execute the file conversion -- train 
prepare_data_generic(path2inputTraining, path2outpuT, grid_size=0.001, verbose=True, protocol="synthetic", 
                     dataset="train")
# Execute the file conversion -- test
prepare_data_generic(path2inputTest, path2outpuT, grid_size=0.001, verbose=True, protocol="synthetic", 
                     dataset="test")
# Note: In the protocol argument you could define different options: 
#       -> synthetic: The data has the XYZ coordinate and the annotations 
#       -> field_only_xyz: The data has the XYZ coordinate and the annotations
#       -> field: The data has the XYZ+radiometric+Annotations -- change

## Training the models 

We will train Random Forest and RandLA-NET to segment the apples in the real point clouds. The data to this test, could be found on the folder "data/example2notebook_realdata". In this folder, could be found 10 annotated real point clouds of apples trees. These point clouds have the XYZ coordinates, the radiometric features and the annotated apples. The steps to train the models are in the following cells. 

### Random Forest 
To train random forest, we are going to split our test in two main groups, "train" that will contain the 80% of the dataset and "test" that will have the final 20%. After do this splitting, we are going to calculate the FPFH features of each group of point clouds. And finally we will launch the trainning over the data.

#### Splitting the data

In [ ]:
# NOTE: This cell can be exectued only in the code 
out2files = "data/example2notebook_realdata/dataToRDF/"
# This method will split and prepare the data "features and desired format"
split_dataSet(data2annotatedApples, out2files, "rdf", verbose=True, protocol="field")

#### Train

In [ ]:
# Train Random Forest 
import myio
# NOTE: The annotations on the field data are in the 7 position, but as we begin from 0, 
# we will said that is the 6 position  
annotationColum  = 6
# Set the paths to the test and train 
path2trainSet = os.path.join(out2files, "train")
path2testSet  = os.path.join(out2files, "test")
# Load the data 
train_set = myio.load_data(path2testSet)# NOTE: This method will load all the file on the referred folder
                                        # it is possible that if you dont have enough RAM the process crash
print("Original shape of the training set: %s" %(str(train_set.shape)))
# Get the annotation in a different array
y_train = np.array([y[annotationColum] for y in train_set]) # Apple is annotated as 1 and other as 0
print("Annotated points: %i" %(len(y_train)))
print("Found classes   : %s" %(np.unique(y_train)))
# Remove the annotation --
train_set = np.delete(train_set, annotationColum, 1)
# Remove the X Y Z coordinates and leave only the Features 
for _ in range(3):
    train_set = np.delete(train_set, 0, 1)
print("Final shape: %s" %(str(train_set.shape))) 

In [ ]:
from machine_learning.RFClassifier import RFClassifier
path2saveModel = os.path.join(data2annotatedApples, "trained_rdf.sav")
# Train the random forest model 
model = RFClassifier()
model.set_train_test_data(train_set, y_train, train_set, y_train)
model.train()
model.save(path2saveModel)

### RandLA-NET

In [ ]:
# Prepare the data to randlanet 
out2files = "data/example2notebook_realdata/dataToRDNET/"
split_dataSet(data2annotatedApples, out2files, "rdnet", verbose=True, protocol="field")

In [ ]:
import sys 
sys.path.append("./utils")
from randlanet.main_apple_tree import *

path2saveModel_r = os.path.join(data2annotatedApples, "trained_randlanet")

param = {"gpu":0, # GPU ID
         "mode":"train",  # to Predict
         "model_path":None, 
         "path2data": out2files, 
         "path2output": path2saveModel_r, # This arg only works to save the training 
         "protocol":"field_only_xyz", 
         "restoreTrain":False}
# RandLA-NET -- Under corrections -- Execute the script from Terminal
train_field(out2files, path2saveModel_r, parameters=param)
#train_field_only_xyz(out2files, path2saveModel_r, parameters=param)
#train_synthetic_HiHiRes(out2files, path2saveModel_r, parameters=param)

# NOTE: If a NoneType error is launched, verify that the point clouds are bigger 
# enough to the subsampling process, if not change the num_points variable in the script 
# helper_tool.py

## Point Clouds segmentation

The segmentation of the fruits over the point clouds it is going to be approached using 3 different methods. The first one will use the FPFH to do a threshold and detect the apples. The second will use the Random Forest to classify the class of the points using the FPFH. Finally we will run the RandLA-NET model over the data.


### Classic Approach 

In [ ]:
# Prepare the data for the classic method 
files_lst = glob.glob(os.path.join(data_fpfh,"*.txt"))
print("Found files: %i" %(len(files_lst)))
sampleCloud = np.loadtxt(files_lst[0])
featuresIni = sampleCloud.shape[1]-1 # We remove 1 because of the annotations.
                                     # Features on the file: 
                                     #   -> XYZ  [3]
                                     #   -> Radiometric Values  [3] [Reflectance, Deviation, Amplitude]
                                     #   -> FPFH [33]
path2wrt = os.path.join(data_fpfh, "noRadiometry/")
if(not os.path.isdir(path2wrt)):
    os.mkdir(path2wrt)
print("Features: %i" %featuresIni)
for fileNamePC in files_lst:
    print("Loading file: %s"%(split_path_and_fileName(fileNamePC)[1]))
    a_pc = np.loadtxt(fileNamePC)
    print("  -> Original shape: %s" %(str(a_pc.shape)))
    for i in range(5,2,-1): # Delete the radiometric columns  
        a_pc = np.delete(a_pc, i, 1)
    fname2wrt = path2wrt+split_path_and_fileName(fileNamePC)[1]
    print("  -> Final shape: %s" %(str(a_pc.shape)))
    print("  -> File is going to  be wrote: %s " %fname2wrt)
    np.savetxt(fname2wrt, a_pc)

In [ ]:
!python pcl/launcher.py --action=1 data/example2notebook_realdata/fpfh/noRadiometry/ data/example2notebook_realdata/fpfh/segmentation_classic/

### Random Forest 

In [ ]:
# Terminal 
!mkdir data/example2notebook_realdata/dataToRDF/predicted/
!python predict.py --model=data/example2notebook_realdata/trained_rdf.sav --path2data=data/example2notebook_realdata/dataToRDF/test/ --path2write=data/example2notebook_realdata/dataToRDF/predicted/

In [ ]:
from predict import synthetic_predict_rf
# Code 
rdf_test = os.path.join(data2annotatedApples, "dataToRDF/test/")
outputRdf = os.path.join(data2annotatedApples, "dataToRDF/predicted/")
path2saveModel = os.path.join(data2annotatedApples, "trained_rdf.sav")
# Random Forest -- notebook 
synthetic_predict_rf(path2saveModel, rdf_test, outputRdf)

### RandLA-NET


In [ ]:
# Terminal 
!python utils/main_apple_tree.py --mode=test --model_path=data/example2notebook_realdata/trained_randlanet/snapshots/snap-XXX  --inputDir=data/example2notebook_realdata/dataToRDNET/
#NOTE: Change the XXX for the name of the last available snapshot

In [ ]:
# Code 
import sys 
sys.path.append("./utils/")
from randlanet.main_apple_tree import *
from randlanet.utils.merge_label_apple import *
# Arguments for the model
param = {"gpu":0, # GPU ID
         "mode":"test",  # to Predict
         "model_path":path2saveModel_r, 
         "path2data": out2files, 
         "path2output": "./", # This arg only works to save the training 
         "protocol":"field_only_xyz", 
         "restoreTrain":True}
# Predict
if(param["protocol"] == "synthetic"):
    train_synthetic_HiHiRes(param["path2data"], param["path2output"], parameters=param)
elif(param["protocol"] == "field_only_xyz"):
    train_field_only_xyz(param["path2data"], param["path2output"], parameters=param)
elif(param["protocol"] == "field"):
    train_field(param["path2data"], param["path2output"], parameters=param)
else:
    print("-> Error: Unknow option")
# Merge label
path2prediction = "test/Log_XXXX-XX-XX_XX-XX-XX/predictions/" # Change the XXXX for the respective date 
                                            
path2data = os.path.join(param["path2data"],"test/")
output = os.path.join(out2files, "prediction/")
merge_pointCloudAndLabels(path2data, "./test/", output)

## Apple Counting 

In [ ]:
file_index = 0
files_annApples = glob.glob(os.path.join(data2annotatedApples, "output_rdf_prediction", "*.txt"))
path, fname = split_path_and_fileName(files_annApples[file_index])
fname = "cluster_%s"%(fname)
# Load the processed file 
output = os.path.join(data2annotatedApples, "output_rdf_prediction",fname)
print("Cluster: %s" %(output))
min_samples = 50
eps = 0.04
pcPrediction = np.loadtxt(files_annApples[0])
# Get the clusters using the predicted labels 
clPrediction = clustering(pcPrediction, min_samples, eps)
# Write the cluster 
np.savetxt(output, clPrediction) # To visualize the clusti

In [ ]:
from post_processing.algorithm import clustering 
eps, minSamples = 0.04, 50

path2write = os.path.join(outputRdf, "clustering/")

if(not os.path.isdir(path2write)):
    os.mkdir(path2write)
    
files_annApples = glob.glob(os.path.join(outputRdf,"*.txt"))
print("Found annotated files: %i" %(len(files_annApples)))
for idx, file2clustering in enumerate(files_annApples, start=1):
    _, actualFileName = split_path_and_fileName(file2clustering)
    print("-> Loading[%i/%i]: %s" %(idx, len(files_annApples), actualFileName))
    pointCloud2cluster = np.loadtxt(file2clustering)
    cluster = clustering(pointCloud2cluster, minSamples, eps)
    print(" -> The file will be written in: %s" %path2write)
    np.savetxt(path2write+actualFileName, cluster)

In [ ]:
# Counting the apples  from the generated clusters 
clusteredClouds = glob.glob(os.path.join(path2write,"*.txt"))
colOfClusters   = 4 # The cluster function will add at the end of the predicted files a column with the clusters
                    # and after it will write the file with the new column in the specified folder
print("Files: %i" %((len(clusteredClouds))))
for fileName in clusteredClouds:
    _, actualFileName = split_path_and_fileName(file2clustering)
    pc = np.loadtxt(fileName)
    apples = len(np.unique(pc[:,colOfClusters]))
    print("File: %s" %actualFileName)
    print(" -> Found apples[clusters]: %i" %apples)

### References
[1] Hu, Qingyong, et al. "RandLA-Net: Efficient semantic segmentation of large-scale point clouds." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2020. <br>
[2] Pedregosa, Fabian, et al. "Scikit-learn: Machine learning in Python." the Journal of machine Learning research 12 (2011): 2825-2830.<br>
[3] Raschka, Sebastian, and Vahid Mirjalili. Python machine learning. Packt Publishing Ltd, 2017.<br>
[4] Rusu, R. B., and S. Cousins. "3D is here: point cloud library." Point Cloud Library http://pointclouds.org/. Accessed January 8 (2021). <br>
[5] Schubert, Erich, et al. "DBSCAN revisited, revisited: why and how you should (still) use DBSCAN." ACM Transactions on Database Systems (TODS) 42.3 (2017): 1-21.<br>